In [49]:
import numpy as np
import os
from sklearn.model_selection import train_test_split

from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard

In [50]:
DATA_ROOT = "MP_Data"
actions = np.array(['Hello', 'Thanks', 'Idle'])
sequences = 30
frames = 30

action_map = {label:num for num, label in enumerate(actions)}

sequences_data = []
labels = []

In [51]:
for action in actions:
    for sequence in range(sequences):
        video_list = []

        for frame in range(frames):
            np_data = np.load(os.path.join(DATA_ROOT, action, str(sequence), f"{str(frame)}.npy"))
            video_list.append(np_data)
        
        sequences_data.append(video_list)
        labels.append(action_map[action])

In [52]:
x = np.array(sequences_data)

y = to_categorical(labels).astype(int)

xtrain, xtest, ytrain, ytest = train_test_split(x, y,
                                                test_size=0.2,
                                                stratify=y)

print("First frame of data looks like:")
print(x[0][0])
print(f"Max value in data: {np.max(x)}")
print(f"Min value in data: {np.min(x)}")

First frame of data looks like:
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Max value in data: 1.0830367803573608
Min value in data: 0.0


In [53]:
model = Sequential()

model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30, 42)))
model.add(LSTM(64, return_sequences=False, activation='relu')) # Reduced from 3 layers to 2
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

c:\Users\PC\Documents\Code\Python\CV\ASL-translator\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [54]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model.fit(xtrain, ytrain, epochs=200, callbacks=[TensorBoard(log_dir='Logs')])

model.save('action.h5')
print("Saved model")

Epoch 1/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 5s 40ms/step - categorical_accuracy: 0.3333 - loss: 1.1102
Epoch 2/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - categorical_accuracy: 0.5278 - loss: 1.0820
Epoch 3/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 0.6250 - loss: 1.0528
Epoch 4/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.5833 - loss: 1.0245
Epoch 5/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - categorical_accuracy: 0.6389 - loss: 0.9609
Epoch 6/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - categorical_accuracy: 0.8333 - loss: 0.8575
Epoch 7/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - categorical_accuracy: 0.6250 - loss: 0.6953
Epoch 8/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - categorical_accuracy: 0.7361 - loss: 0.6364
Epoch 9/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 0.8472 - loss: 0.5568
Epoch 10/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - categorical_accuracy: 0.8194 - loss: 0.5442
Epoch 11/200
3/3 ━━━━━━━━━━━━━━━━━━━━ 0

Saved model
